In [1]:
from keras_preprocessing import image
import pandas as pd
import numpy as np 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, Dropout
from keras.models import load_model
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
datagen_train = image.ImageDataGenerator(rescale=1./255, 
                                         rotation_range=10, 
                                         width_shift_range = 0.2, 
                                         height_shift_range = 0.2, 
                                         shear_range = 0.2, zoom_range = 0.2, 
                                         fill_mode = "nearest", validation_split=0.20)

datagen_test = image.ImageDataGenerator(rescale=1./255)

traindf = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
traindf['category'] = traindf['category'].astype('str')

training = datagen_train.flow_from_dataframe(dataframe=traindf,
                                             directory="./train/",
                                             x_col="image",
                                             y_col='category',
                                             target_size = (120,120),
                                             class_mode = "categorical",
                                             subset='training',
                                             seed=42,shuffle=True)

validation = datagen_train.flow_from_dataframe(dataframe=traindf,directory="./train/",
                                               x_col="image",y_col='category',
                                               target_size = (120,120), class_mode = "categorical", subset='validation',
                                               seed=42, shuffle=True)

testing = datagen_test.flow_from_dataframe(dataframe=testdf, directory="./test/", x_col="image", y_col=None,
                                           target_size = (120,120),
                                           batch_size = 28,
                                           class_mode = None, seed=42, shuffle=False)

Found 5002 validated image filenames belonging to 5 classes.
Found 1250 validated image filenames belonging to 5 classes.
Found 2680 validated image filenames.


In [33]:
import keras
def conv2D_model():
    
    model = Sequential()
    
    model.add(Conv2D(filters=1024, kernel_size=(3,3), padding = 'same',
                input_shape=(120,120,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) #40x40
    model.add(Dropout(0.25))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) #20x20
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) #10x10
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (10, 10), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) #5x5
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (10, 10), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2))) #5x5
    model.add(Dropout(0.25))
    
#     model.add(Conv2D(32, (10, 10), padding='same', activation='relu'))
#     model.add(Dropout(0.25))
    
#     model.add(Conv2D(32, (10, 10), padding='same', activation='relu'))
#     model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (10, 10), padding='same', activation='relu'))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(5, activation='softmax'))
    # sgd = SGD(lr=0.03, decay=1e-4, momentum=0.9, nesterov=True)
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


    STEP_SIZE_TRAIN = training.n // training.batch_size
    STEP_SIZE_VALID = validation.n // validation.batch_size

    
    model.fit_generator(generator=training,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=10,
    validation_data=validation,
    validation_steps=STEP_SIZE_VALID)
    
    evaluator = model.evaluate_generator(generator=validation, steps=STEP_SIZE_VALID)
    
    return model

In [ ]:
model = conv2D_model()

print('Saving the model...')

model.save('best_model.h5')

print('Model Succesfully Trained.')

Epoch 1/20
  1/156 [..............................] - ETA: 5:18:35 - loss: 1.6094 - accuracy: 0.1562

In [ ]:
testing.reset()

# model = load_model('best_model.h5')

STEP_SIZE_TEST = testing.n // testing.batch_size
predictions = model.predict_generator(testing, steps=STEP_SIZE_TEST, verbose=1)

predicted_class_indices = np.argmax(predictions, axis = 1)

labels = training.class_indices
labels = {value:key for key, value in labels.items()}
predictions = [labels[value] for value in predicted_class_indices]
print(len(predictions))
print(predictions)

filenames = [filename for filename in testing.filenames]
print(len(filenames))
# results = pd.DataFrame({"Filename":filenames, "Predictions":predictions}, index = None)
# results.to_csv("Results.csv", index=False)
# print('Results Saved !!')